<a href="https://colab.research.google.com/github/LostUniavan/ScikitLearn/blob/main/ScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import csv
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

def prepare_csv(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        for i, row in enumerate(reader):
            label = i
            row.append(label)
            writer.writerow(row)

def apply_visual_noise(data, noise_level):
    noisy_data = data.copy()
    num_elements = noisy_data.size

    ones_indices = np.where(noisy_data == 1)[0]
    zeros_indices = np.where(noisy_data == 0)[0]

    num_noisy_ones = int(len(ones_indices) * noise_level)
    num_noisy_zeros = int(len(zeros_indices) * noise_level)

    noisy_ones_indices = np.random.choice(ones_indices, num_noisy_ones, replace=False)
    noisy_zeros_indices = np.random.choice(zeros_indices, num_noisy_zeros, replace=False)

    for index in noisy_ones_indices:
        noisy_data.flat[index] = 0
    for index in noisy_zeros_indices:
        noisy_data.flat[index] = 1

    return noisy_data

def evaluate_mlp_with_visual_noise(mlp, test_data, expected_outputs, noise_levels):
    for noise_level in noise_levels:
        noisy_inputs = np.array([apply_visual_noise(x, noise_level) for x in test_data])
        predictions = mlp.predict(noisy_inputs)
        accuracy = accuracy_score(expected_outputs, predictions)
        correct = int(accuracy * len(test_data))
        incorrect = len(test_data) - correct
        print(f"Nível de ruído: {noise_level*100}%, Correto: {correct}, Incorreto: {incorrect}")

input_file = 'data_input.csv'
output_file = 'data_input_preparado.csv'
prepare_csv(input_file, output_file)

data = np.loadtxt(output_file, delimiter=',')
inputs = data[:, :-1]
expected_outputs = data[:, -1]

mlp = MLPClassifier(hidden_layer_sizes=(64, 32), activation='tanh', solver='adam', learning_rate_init=0.01, max_iter=5000)
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
print(f"Treinamento começou em: {start_time}")
mlp.fit(inputs, expected_outputs)
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
print(f"Treinamento finalizado em {mlp.n_iter_} epochs. Com um erro final de {mlp.loss_}")
print(f"Treinamento finalizado em: {end_time}")

predictions = mlp.predict(inputs)
accuracy = accuracy_score(expected_outputs, predictions)
correct = int(accuracy * len(inputs))
incorrect = len(inputs) - correct
print(f"Teste com os dados de treinamento vs resultado esperado, Correto: {correct}, Incorreto: {incorrect}")

noise_levels = [0.02, 0.05, 0.1, 0.3]
evaluate_mlp_with_visual_noise(mlp, inputs, expected_outputs, noise_levels)


Treinamento começou em: 2024-07-05 00:46:15
Treinamento finalizado em 190 epochs. Com um erro final de 0.11286223900281867
Treinamento finalizado em: 2024-07-05 00:46:17
Teste com os dados de treinamento vs resultado esperado, Correto: 203, Incorreto: 7
Nível de ruído: 2.0%, Correto: 200, Incorreto: 10
Nível de ruído: 5.0%, Correto: 191, Incorreto: 19
Nível de ruído: 10.0%, Correto: 137, Incorreto: 73
Nível de ruído: 30.0%, Correto: 16, Incorreto: 194
